<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s Overal Message:
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [118]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [104]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-09-01 05:39:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-01 05:39:57 (80.9 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



In [105]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [106]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-09-01 05:39:57--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 173.194.215.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.3’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-01 05:39:57 (139 MB/s) - ‘nlp_getting_started.zip.3’ saved [607343/607343]



In [107]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [108]:
train_df = pd.read_csv('train.csv')
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [119]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [110]:
test_df = pd.read_csv('test.csv')
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [111]:
random_data = train_df.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
#np Avenged Sevenfold - Hail To The King : This is NOT Disaster tweet
------------------------------------

tweet number 2 :
@_minimehh @cjoyner I must be overlooking the burning buildings? #BlackLivesMatter : This is a Disaster tweet
------------------------------------

tweet number 3 :
incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring : This is a Disaster tweet
------------------------------------

tweet number 4 :
#SBNation #Flames What Makes a Good Penalty Killer? http://t.co/xYi5fDacxO http://t.co/SjtvzgGcXU : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
@thoutaylorbrown I feel like accidents are just drawn to you but I'm happy you've survived all of them. Hope you're okay!!! : This is NOT Disaster tweet
------------------------------------



## Extract validation data from train data:

In [122]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [101]:
train_df['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [128]:
train_data, val_data, train_label, val_label = train_test_split(train_df['text'].to_numpy(), train_df['target'].to_numpy(), 
                                                                test_size=0.1, 
                                                                random_state=42)

In [155]:
random_num = np.random.choice(len(train_data),size=761,replace=False)
random_num

array([ 542, 6449, 6426, 5114, 4572, 3338, 6195, 2190, 2221,  381, 4206,
       2150,  269, 4252,  555, 2218,  545,  130, 6819, 6756, 1793, 6269,
       1903, 2573, 5765, 2068, 1894, 2287, 5828, 3090,   97, 5639, 4715,
       2975, 4153, 2777, 2521,  436, 1860, 6240, 6055, 3237, 3174, 2324,
       1266, 1019, 3279,  672, 2371, 4256, 4205, 5851,  504, 2596, 2852,
       5212, 3651, 3974, 3957, 3198, 6712, 3776, 2911, 1677, 5013, 2352,
       3263, 3649, 1256, 3012, 6128, 5321, 1558, 2648,  114, 4360, 5055,
       3377, 3485, 5935, 5611, 5328, 1035, 3985,  430, 2514, 3324, 6666,
       6491, 3186, 4769, 6152,  523, 2881, 5869, 1730, 3192, 2155, 1326,
       4449, 6067, 6768, 3860, 5915, 1288, 1559, 2231, 5305, 4868, 1032,
       4536, 3648, 3834, 1539, 4652, 2769, 4599, 6590, 5783,  239, 1197,
        444, 6658, 1410, 5721, 4657, 4414, 5823,  232, 4940, 4165, 2076,
        333, 6679, 4514, 6317, 4194, 2077, 4223, 1960,  373, 6688, 4606,
       6608,  922, 2052, 5575, 3855, 4920,  324, 38

In [158]:
train_label_10_percent = train_label[random_num]

In [159]:
train_data_10_percent = train_data[random_num]

In [44]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [45]:
sum([len(tweet.split()) for tweet in train_data])

101905

In [46]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [47]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [48]:
text_vectorizer.adapt(train_data)

In [49]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  707,    9,    1,    8,  171,  714,    5, 1815,   12,    0,
          0,    0,    0,    0])>

In [50]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  when things become terrible more than the great Ethiopian famine... 


vectorized tweet:  tf.Tensor(
[  45  380  822 1964   50   78    2  191 5651  486    0    0    0    0
    0], shape=(15,), dtype=int64)


In [51]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['palm', 'palinfoen', 'palestinian\x89Û', 'palermo', 'pale', 'pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


## Embedding:

In [52]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim =32, input_length=None)

In [53]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[-0.00141915,  0.00370733,  0.03474445, -0.04192179, -0.02048619,
        -0.00869516,  0.04004619, -0.03916598,  0.0157127 ,  0.01956853,
        -0.02866615,  0.03803643, -0.02646535, -0.0047574 , -0.0251094 ,
        -0.01520016, -0.03481402, -0.02327779,  0.00966324, -0.00404912,
         0.00881076, -0.04248139,  0.00235955, -0.03617594,  0.04347483,
        -0.02485937, -0.02984872,  0.02158963, -0.03088179, -0.03307908,
        -0.0257238 , -0.02629236],
       [ 0.01818227,  0.03268112,  0.04193315,  0.04428772, -0.02928338,
        -0.00459876, -0.03770386,  0.01450906, -0.00532839, -0.01146779,
        -0.02845788,  0.01317834, -0.00816841,  0.00548853,  0.04323598,
        -0.02623789,  0.02908159, -0.01549195,  0.00700921,  0.04684554,
         0.00412595,  0.03450534,  0.036614  , -0.03646196, -0.00582768,
        -0.02896266, -0.03818249,  0.00480633, -0.04022126, -0.00610018,
        -0.01367217, -0.02976644],
    

## Analyze with Naive Bayes using TF-IDF:

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [55]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [56]:
val_pred = model_0.predict(val_data)
val_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [57]:
val_label[:10]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 1])

In [58]:
accuracy_score(val_label, val_pred)

0.7782152230971129

In [59]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.74      0.92      0.82       426
           1       0.86      0.60      0.70       336

    accuracy                           0.78       762
   macro avg       0.80      0.76      0.76       762
weighted avg       0.79      0.78      0.77       762



In [60]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [61]:
model_0.predict_proba(tweets)

array([[0.22063312, 0.77936688],
       [0.58453661, 0.41546339],
       [0.71572258, 0.28427742]])

In [62]:
model_0.predict(tweets)

array([1, 0, 0])

## Analyze with Vanilla Neural Network:

In [63]:
from helper_functions import create_tensorboard_callback

In [84]:
# sequential API

# model_1 = tf.keras.Sequential([
#     tf.keras.layers.Input(shape= (1,), dtype = tf.string),
#     text_vectorizer,
#     embedding,
#     tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')    
# ])

# model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
#                  optimizer = tf.keras.optimizers.Adam(),
#                  metrics= ['accuracy'])

In [165]:
# Functional API

inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [166]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d_9   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_18 (Dense)            (None, 10)                330       
                                                                 
 dense_19 (Dense)            (None, 1)                 11      

In [167]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [168]:
# !rm -rf './tensorboard/ANN'

In [171]:
model_1.fit(train_data_10_percent, 
            train_label_10_percent, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220901-062035
Epoch 1/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0439 - accuracy: 0.9803 - val_loss: 1.0258 - val_accuracy: 0.7375
Epoch 2/10
24/24 [==============================] - 0s 10ms/step - loss: 0.0345 - accuracy: 0.9908 - val_loss: 1.0437 - val_accuracy: 0.7323
Epoch 3/10
24/24 [==============================] - 0s 11ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 1.0485 - val_accuracy: 0.7310
Epoch 4/10
24/24 [==============================] - 0s 11ms/step - loss: 0.0259 - accuracy: 0.9934 - val_loss: 1.0509 - val_accuracy: 0.7336
Epoch 5/10
24/24 [==============================] - 0s 10ms/step - loss: 0.0234 - accuracy: 0.9934 - val_loss: 1.0535 - val_accuracy: 0.7349
Epoch 6/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0215 - accuracy: 0.9947 - val_loss: 1.0657 - val_accuracy: 0.7349
Epoch 7/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0202 - accurac

In [170]:
history_1 = model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220901-062004
Epoch 1/10
215/215 [==============================] - 2s 8ms/step - loss: 0.1730 - accuracy: 0.9485 - val_loss: 0.5580 - val_accuracy: 0.7651
Epoch 2/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1232 - accuracy: 0.9587 - val_loss: 0.6345 - val_accuracy: 0.7533
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1004 - accuracy: 0.9648 - val_loss: 0.7081 - val_accuracy: 0.7415
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0865 - accuracy: 0.9699 - val_loss: 0.7571 - val_accuracy: 0.7493
Epoch 5/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0772 - accuracy: 0.9730 - val_loss: 0.8133 - val_accuracy: 0.7441
Epoch 6/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0696 - accuracy: 0.9745 - val_loss: 0.8624 - val_accuracy: 0.7415
Epoch 7/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0638 - 

In [ ]:
model_1.evaluate(val_data,val_label)

24/24 [==============================] - 0s 3ms/step - loss: 0.8235 - accuracy: 0.7402


[0.8235107064247131, 0.7401574850082397]

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_1.predict(tweets)

array([[0.9784072 ],
       [0.7917726 ],
       [0.08460793]], dtype=float32)

## Retrieve the trained word embeddings:

In [ ]:
weights = model_1.get_layer('embedding').get_weights()[0]
weights

array([[ 0.25929037, -0.06279013, -0.06148194, ...,  0.14954968,
         0.10060261,  0.0653062 ],
       [ 0.24403132, -0.03507703, -0.0686942 , ...,  0.12375301,
         0.10604884,  0.02397103],
       [ 0.26330766, -0.00792165,  0.00596493, ...,  0.15860574,
         0.17530532,  0.0099184 ],
       ...,
       [ 0.08522397,  0.05309212,  0.0784494 , ...,  0.00166323,
        -0.00641821, -0.00086374],
       [ 0.01429316,  0.00259889, -0.01726699, ..., -0.02082995,
        -0.01213987, -0.01905471],
       [ 0.04047877, -0.02340493, -0.01548165, ...,  0.00393052,
         0.012569  , -0.03269402]], dtype=float32)

In [ ]:
weights.shape

(10000, 32)

In [ ]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Then, use [Embedding Projector](http://projector.tensorflow.org/) to Visualize the embeddings

## Analyze with LSTM model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.LSTM(64,  return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs, name='LSTM')

In [ ]:
model_2.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            24832     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                              

In [ ]:
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/LSTM'

In [ ]:
history_2 = model_2.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','LSTM')])

Saving TensorBoard log files to: tensorboard/LSTM/20220831-192716
Epoch 1/10
215/215 [==============================] - 11s 30ms/step - loss: 0.1997 - accuracy: 0.9218 - val_loss: 0.8313 - val_accuracy: 0.7507
Epoch 2/10
215/215 [==============================] - 6s 26ms/step - loss: 0.0811 - accuracy: 0.9685 - val_loss: 0.8949 - val_accuracy: 0.7388
Epoch 3/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0639 - accuracy: 0.9749 - val_loss: 1.2104 - val_accuracy: 0.7441
Epoch 4/10
215/215 [==============================] - 6s 27ms/step - loss: 0.0608 - accuracy: 0.9737 - val_loss: 1.0866 - val_accuracy: 0.7178
Epoch 5/10
215/215 [==============================] - 6s 26ms/step - loss: 0.0509 - accuracy: 0.9772 - val_loss: 1.2095 - val_accuracy: 0.7362
Epoch 6/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0468 - accuracy: 0.9793 - val_loss: 1.5439 - val_accuracy: 0.7362
Epoch 7/10
215/215 [==============================] - 5s 25ms/step - loss: 

In [ ]:
model_2.evaluate(val_data,val_label)

24/24 [==============================] - 0s 7ms/step - loss: 1.8930 - accuracy: 0.7139


[1.8929849863052368, 0.7139107584953308]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_2.predict(tweets)

array([[0.9999676 ],
       [0.9162924 ],
       [0.00160667]], dtype=float32)

## Analyze with GRU model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.GRU(64,  return_sequences=True)(x)
x = tf.keras.layers.GRU(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs, name='GRU')

In [ ]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 gru (GRU)                   (None, 15, 64)            18816     
                                                                 
 gru_1 (GRU)                 (None, 32)                9408      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                               

In [ ]:
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_3 = model_3.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','GRU')])

Saving TensorBoard log files to: tensorboard/GRU/20220831-192822
Epoch 1/10
215/215 [==============================] - 11s 29ms/step - loss: 0.1503 - accuracy: 0.9431 - val_loss: 0.9846 - val_accuracy: 0.7087
Epoch 2/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0608 - accuracy: 0.9781 - val_loss: 0.8491 - val_accuracy: 0.7205
Epoch 3/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0526 - accuracy: 0.9785 - val_loss: 1.0873 - val_accuracy: 0.7493
Epoch 4/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0416 - accuracy: 0.9822 - val_loss: 1.8826 - val_accuracy: 0.7152
Epoch 5/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0450 - accuracy: 0.9812 - val_loss: 1.4369 - val_accuracy: 0.7283
Epoch 6/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0381 - accuracy: 0.9813 - val_loss: 1.6457 - val_accuracy: 0.7283
Epoch 7/10
215/215 [==============================] - 5s 25ms/step - loss: 0

In [ ]:
model_3.evaluate(val_data,val_label)

24/24 [==============================] - 0s 6ms/step - loss: 1.9038 - accuracy: 0.7415


[1.903810977935791, 0.7414698004722595]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_3.predict(tweets)

array([[9.9997842e-01],
       [9.9813539e-01],
       [7.9515576e-04]], dtype=float32)

## Analyzie with Bidirectional LSTM model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)


x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

In [ ]:
model_4.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                

In [ ]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_4 = model_4.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-192923
Epoch 1/10
215/215 [==============================] - 18s 45ms/step - loss: 0.1011 - accuracy: 0.9629 - val_loss: 1.1875 - val_accuracy: 0.7454
Epoch 2/10
215/215 [==============================] - 8s 37ms/step - loss: 0.0400 - accuracy: 0.9818 - val_loss: 1.4493 - val_accuracy: 0.7205
Epoch 3/10
215/215 [==============================] - 8s 38ms/step - loss: 0.0398 - accuracy: 0.9803 - val_loss: 1.3763 - val_accuracy: 0.7467
Epoch 4/10
215/215 [==============================] - 8s 38ms/step - loss: 0.0375 - accuracy: 0.9809 - val_loss: 1.6042 - val_accuracy: 0.7441
Epoch 5/10
215/215 [==============================] - 9s 44ms/step - loss: 0.0369 - accuracy: 0.9813 - val_loss: 1.8181 - val_accuracy: 0.7310
Epoch 6/10
215/215 [==============================] - 8s 37ms/step - loss: 0.0357 - accuracy: 0.9822 - val_loss: 1.6126 - val_accuracy: 0.7441
Epoch 7/10
215/215 [==============================] - 8s 37ms

In [ ]:
model_4.evaluate(val_data,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 1.5350 - accuracy: 0.7349


[1.5350419282913208, 0.7349081635475159]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_4.predict(tweets)

array([[7.7287161e-01],
       [4.7660515e-01],
       [7.0527196e-04]], dtype=float32)

## Analyze with Convolutional 1D model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)
print(x.shape)

x = tf.keras.layers.Conv1D(filters=64, kernel_size=3,padding= 'valid')(x)
print(x.shape)
x = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding= 'valid')(x)
print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

(None, 15, 32)
(None, 13, 64)
(None, 11, 128)


In [ ]:
model_5.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 13, 64)            6208      
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 128)           24704     
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                       

In [ ]:
model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_5 = model_5.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-193057
Epoch 1/10
215/215 [==============================] - 3s 12ms/step - loss: 0.1365 - accuracy: 0.9499 - val_loss: 0.9928 - val_accuracy: 0.7349
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0646 - accuracy: 0.9740 - val_loss: 1.2160 - val_accuracy: 0.7192
Epoch 3/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0466 - accuracy: 0.9774 - val_loss: 1.4520 - val_accuracy: 0.7362
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0400 - accuracy: 0.9804 - val_loss: 1.6313 - val_accuracy: 0.7362
Epoch 5/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0388 - accuracy: 0.9812 - val_loss: 1.8688 - val_accuracy: 0.7231
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0356 - accuracy: 0.9799 - val_loss: 1.9609 - val_accuracy: 0.7244
Epoch 7/10
215/215 [==============================] - 2s 11ms/

In [ ]:
model_5.evaluate(val_data,val_label)

24/24 [==============================] - 0s 3ms/step - loss: 2.6490 - accuracy: 0.7034


[2.6489670276641846, 0.7034120559692383]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_5.predict(tweets)

array([[0.99974126],
       [0.9929791 ],
       [0.22827303]], dtype=float32)

## Analyze with Universal Sentence Encoder tensorflow hub:

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed(["The quick brown fox jumps over the lazy dog.",
                      "I am a sentence for which I would like to get its embedding"])

print(embed_sample)

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.05080861 -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [ ]:
embed(train_data)

<tf.Tensor: shape=(6851, 512), dtype=float32, numpy=
array([[ 0.01178656, -0.00748094,  0.00952065, ...,  0.04002216,
        -0.07293098,  0.01045   ],
       [ 0.08340861, -0.03587737,  0.01278975, ..., -0.00764408,
        -0.06952015, -0.08435119],
       [ 0.03161547,  0.01725737, -0.00808266, ..., -0.03697908,
         0.03299483, -0.04934364],
       ...,
       [-0.03393723, -0.06167451,  0.04978931, ..., -0.05219645,
        -0.05812604, -0.0630734 ],
       [ 0.0215332 ,  0.06060665, -0.07061633, ..., -0.04428503,
        -0.00652964, -0.00850323],
       [ 0.02649508, -0.05326023, -0.05051778, ..., -0.01227468,
         0.01156288, -0.00953933]], dtype=float32)>

In [ ]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               trainable=False,
                               dtype=tf.string,
                               name= 'USE')

In [ ]:
model_6 = tf.keras.Sequential([
    encoder_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')],
    name = 'USE_pretrained_model'
)

In [ ]:
model_6.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 34,945
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_6 = model_6.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','USE_pretrained_model')])

Saving TensorBoard log files to: tensorboard/USE_pretrained_model/20220831-193202
Epoch 1/10
215/215 [==============================] - 7s 16ms/step - loss: 0.4908 - accuracy: 0.7723 - val_loss: 0.4170 - val_accuracy: 0.8189
Epoch 2/10
215/215 [==============================] - 4s 17ms/step - loss: 0.4066 - accuracy: 0.8229 - val_loss: 0.4053 - val_accuracy: 0.8176
Epoch 3/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3838 - accuracy: 0.8355 - val_loss: 0.4001 - val_accuracy: 0.8176
Epoch 4/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3628 - accuracy: 0.8441 - val_loss: 0.4013 - val_accuracy: 0.8163
Epoch 5/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3424 - accuracy: 0.8556 - val_loss: 0.3971 - val_accuracy: 0.8228
Epoch 6/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3134 - accuracy: 0.8720 - val_loss: 0.4126 - val_accuracy: 0.8228
Epoch 7/10
215/215 [==============================] - 3s 12m

In [ ]:
model_6.evaluate(val_data,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 0.4979 - accuracy: 0.8031


[0.4979403018951416, 0.8031495809555054]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_6.predict(tweets)

array([[0.9984745 ],
       [0.9650059 ],
       [0.00851494]], dtype=float32)

## Finding most wrong predicted tweets:

In [ ]:
train_data

array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
       'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
       "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
       ...,
       'Omron HEM-712C Automatic Blood Pressure Monitor STANDARD AND LARGE BP CUFFS http://t.co/gJBAInQWN9 http://t.co/jPhgpL1c5x',
       'Officials say a quarantine is in place at an Alabama home over a possible Ebola case after developing symptoms... http://t.co/rqKK15uhEY',
       'I moved to England five years ago today. What a whirlwind of time it has been! http://t.co/eaSlGeA1B7'],
      dtype=object)

In [ ]:
train_label

array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
import numpy as np
pred_probs = np.squeeze(model_6.predict(train_data))
print(pred_probs)
pred_label = np.squeeze(np.round(pred_probs))
pred_label

[0.15293044 0.79391134 0.01531976 ... 0.01272947 0.9971067  0.89760965]


array([0., 1., 0., ..., 0., 1., 1.], dtype=float32)

In [ ]:
import pandas as pd
comparison_table = pd.DataFrame({'tweets':train_data,
              'target': train_label,
              'pred': pred_label,
              'prob': pred_probs})

comparison_table

,tweets,target,pred,prob
0,'McFadden Reportedly to Test Hamstring Thursda...,0,0.0,0.152930
1,w--=-=-=-[ NEMA warns Nigerians to prepare for...,1,1.0,0.793911
2,When I was cooking earlier I got electrocuted ...,0,0.0,0.015320
3,I'm On Fire. http://t.co/WATsmxYTVa,0,0.0,0.102810
4,More than 40 families affected by the fatal ou...,1,1.0,0.953237
...,...,...,...,...
6846,@Eganator2000 There aren't many Obliteration s...,0,0.0,0.303111
6847,just had a panic attack bc I don't have enough...,0,0.0,0.012939
6848,Omron HEM-712C Automatic Blood Pressure Monito...,0,0.0,0.012729
6849,Officials say a quarantine is in place at an A...,1,1.0,0.997107


In [ ]:
worst_preds = comparison_table[comparison_table['target'] != comparison_table['pred']].sort_values('prob')
worst_preds

,tweets,target,pred,prob
4418,The media needs to stop publicizing mass murde...,1,0.0,0.005087
1507,OH MY GOD RYANS IN TROUBLE http://t.co/ADIp0UnXHU,1,0.0,0.009996
231,Omg if Cain dies i will be an emotional wreck ...,1,0.0,0.023653
2189,ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate ...,1,0.0,0.030777
3040,#Nursing alert: Emergency Department Psychiatr...,1,0.0,0.031587
...,...,...,...,...
243,@adorableappple No reported flooding po in the...,0,1.0,0.803092
5890,Teen Disaster Preparedness Event in Van Nuys A...,0,1.0,0.821565
4648,wowo--=== 12000 Nigerian refugees repatriated ...,0,1.0,0.827007
4653,#hot C-130 specially modified to land in a st...,0,1.0,0.938754


In [ ]:
worst_preds[:10]

,tweets,target,pred,prob
4418,The media needs to stop publicizing mass murde...,1,0.0,0.005087
1507,OH MY GOD RYANS IN TROUBLE http://t.co/ADIp0UnXHU,1,0.0,0.009996
231,Omg if Cain dies i will be an emotional wreck ...,1,0.0,0.023653
2189,ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate ...,1,0.0,0.030777
3040,#Nursing alert: Emergency Department Psychiatr...,1,0.0,0.031587
2320,Why are you deluged with low self-image? Take ...,1,0.0,0.031758
2360,Detonation fashionable mountaineering electron...,1,0.0,0.032980
3845,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.037642
523,Do you feel like you are sinking in low self-i...,1,0.0,0.041536
5014,?? New Ladies Shoulder Tote #Handbag Faux Leat...,1,0.0,0.041920


In [ ]:
worst_preds[-10:]

,tweets,target,pred,prob
4517,Hollywood Movie About Trapped Miners Released ...,0,1.0,0.770442
2335,@mickinyman @TheAtlantic That or they might be...,0,1.0,0.781370
5859,.POTUS #StrategicPatience is a strategy for #G...,0,1.0,0.783202
1879,FWD: I literally jumped out of bed put on beac...,0,1.0,0.784837
216,Video Captures Man Removing American Flag From...,0,1.0,0.802437
243,@adorableappple No reported flooding po in the...,0,1.0,0.803092
5890,Teen Disaster Preparedness Event in Van Nuys A...,0,1.0,0.821565
4648,wowo--=== 12000 Nigerian refugees repatriated ...,0,1.0,0.827007
4653,#hot C-130 specially modified to land in a st...,0,1.0,0.938754
5582,Kosciusko police investigating pedestrian fata...,0,1.0,0.994299


### 5 most wrong disaster tweets:


In [ ]:
for i in np.arange(0,5):
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

The media needs to stop publicizing mass murder. So many sick people do these things for the eyes of the world &amp; the media is not helping. 

target: 1, prediction: 0.0, prob: 0.005086719989776611 

OH MY GOD RYANS IN TROUBLE http://t.co/ADIp0UnXHU 

target: 1, prediction: 0.0, prob: 0.009995579719543457 

Omg if Cain dies i will be an emotional wreck #emmerdale 

target: 1, prediction: 0.0, prob: 0.02365279197692871 

ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate snow!

Me: Lol u call this a snowstorm..?

#growingupincoloradoÛ 

target: 1, prediction: 0.0, prob: 0.03077659010887146 

#Nursing alert: Emergency Department Psychiatric RN (.90 FTE Day) | Providence Health &amp; Services | #Olympia WA http://t.co/Yu6NUe7gFB 

target: 1, prediction: 0.0, prob: 0.03158724308013916 



### 5 most wrong Non-disaster tweets:


In [ ]:
for i in np.arange(1,6)*-1:
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

Kosciusko police investigating pedestrian fatality hit by a train Thursday http://t.co/m5djLLxoZP 

target: 0, prediction: 1.0, prob: 0.9942992329597473 

#hot  C-130 specially modified to land in a stadium and rescue hostages in Iran in 1980 http://t.co/zY3hpdJNwg #prebreak #best 

target: 0, prediction: 1.0, prob: 0.9387543201446533 

wowo--=== 12000 Nigerian refugees repatriated from Cameroon 

target: 0, prediction: 1.0, prob: 0.8270071148872375 

Teen Disaster Preparedness Event in Van Nuys August 11 @ 5:30pm http://t.co/fXUX987vZx via @VanNuysCouncil 

target: 0, prediction: 1.0, prob: 0.8215647339820862 

@adorableappple No reported flooding po in the area. Ten-4. #mmda 

target: 0, prediction: 1.0, prob: 0.8030920624732971 

